In [1]:
import pandas as pd
import numpy as np
import torch
import os
import torch.optim as optim
import matplotlib.pyplot as plt
import torchmetrics

from tqdm import tqdm
from torchvision import transforms
from PIL import Image
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler


In [2]:
device = ('cuda:0' if torch.cuda.is_available() else 'cpu')

In [3]:
# This class will be used to load the dataset and preprocess it
class HouseDataset(Dataset):
    def __init__(self, csv_file, image_folder, drop_col=None):
        super(HouseDataset, self).__init__()
        self.image_folder = image_folder
        self.drop_col = drop_col

        df = self.load_df(csv_file) #load the text file
        self.features, self.labels = self.preprocess_data(df) # call the preprocess function to preprocess the data
        transform_list = [
            transforms.Grayscale(1),
            transforms.Resize((64, 64)), 
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,)) 
        ] # this is the list of transforms that will be applied to the images
        self.transform = transforms.Compose(transform_list)

    # this function will load the text file and return a dataframe
    def load_df(self, csv_file):
        df_list = pd.read_csv(csv_file).values.tolist() # read the text file and convert it to a list
        data = [line[0].split(' ') for line in df_list]
        for i in range(len(data)):
            data[i] = [(float(x) if '.' in x else int(x)) for x in data[i] if x != ''] # convert the string to float or int
        df = pd.DataFrame(data, index=range(1, len(data) + 1),
                          columns=['Bedrooms', 'Bathrooms', 'Area', 'Zipcode', 'Prices']) # convert the list to a dataframe and add the column names
        self.seq_len = df.shape[1]-1 # extract the number of features
        return df

    def preprocess_data(self, df):
        # Process the Zipcode and Prices
        # get the unique zipcodes to then map them to a number by order then replace the zipcode column with the new numbers
        zipcode_unique = df['Zipcode'].unique()
        zipcode_dict = dict(zip(zipcode_unique, range(len(zipcode_unique))))
        df['Zipcode'] = df['Zipcode'].map(zipcode_dict)
        
        # get the unique prices to then make a list of boundaries to categorize the prices then replace the prices column with the new categories
        df['Prices'] = df['Prices'] / 100000
        prices_boundaries = list(np.arange(int(min(df['Prices'])), int(max(df['Prices'])), 10))
        prices_boundaries.append(np.inf)
        labels = list(range(len(prices_boundaries) - 1))
        df['PriceCategory'] = pd.cut(df['Prices'], bins=prices_boundaries, labels=labels, include_lowest=True) 
        
        # Splitting the features and labels
        features = df.drop('PriceCategory', axis=1).drop('Prices', axis=1)
        if self.drop_col: 
            features = features.drop(self.drop_col, axis=1) # this will drop the column that is passed in the constructor
        labels = torch.tensor(df['PriceCategory'].values, dtype=torch.long)
        
        # Standardize the features so we make sure that all the features are in the same range  (mean = 0, std = 1)
        scaler = StandardScaler()
        features = pd.DataFrame(scaler.fit_transform(features), columns=features.columns)

        return features, labels

    def __len__(self):
        return len(self.features)


    def __getitem__(self, idx):
        # Getting images
        image_names = [f"{idx+1}_bathroom.jpg", f"{idx+1}_bedroom.jpg", f"{idx+1}_frontal.jpg", f"{idx+1}_kitchen.jpg"]
        images = [Image.open(f"{self.image_folder}/{name}") for name in image_names]
        images = [self.transform(image) for image in images]
        images = [torch.tensor(image) for image in images]
        # Getting textual features
        textual_data = torch.tensor(self.features.iloc[idx].values, dtype=torch.float32)
        textual_features = torch.tensor(textual_data)
        # Getting labels
        label = self.labels[idx]

        return images, textual_features, label

In [9]:
# Instantiate the dataset
# NOTE: uncomment the dataset which excludes the column that you want to drop
all_dataset = HouseDataset(csv_file='Data/HousesInfo.txt', image_folder='Data/Dataset')
# drop_zc = HouseDataset(csv_file='Data/HousesInfo.txt', image_folder='Data/Dataset', drop_col='Zipcode')
# drop_rn = HouseDataset(csv_file='Data/HousesInfo.txt', image_folder='Data/Dataset', drop_col='Bedrooms')
# drop_ba = HouseDataset(csv_file='Data/HousesInfo.txt', image_folder='Data/Dataset', drop_col='Bathrooms')
# drop_ar = HouseDataset(csv_file='Data/HousesInfo.txt', image_folder='Data/Dataset', drop_col='Area')

# pass the dataset you chose to the main dataset parameter which will then be used in the dataloader
dataset=all_dataset
# dataset=drop_zc
# dataset=drop_rn
# dataset=drop_ba
# dataset=drop_ar


In [10]:
# Split the dataset into train, validation and test and create the dataloaders

def split_data(dataset, batch_size=32):
    # Define the size of each split of each dataset
    train_split = int(0.65*len(dataset))
    test_split = int(0.2*len(dataset))
    val_split = len(dataset) - train_split - test_split
    # Use the random_split function to split dataset into non-overlapping datasets of the given lengths
    train_data, val_data, test_data = random_split(dataset, [train_split, val_split, test_split])
    
    # Create the dataloaders 
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True) # we shuffle the training data to make sure batches will be different in each epoch
    val_loader = DataLoader(val_data, batch_size=batch_size)
    test_loader = DataLoader(test_data, batch_size=batch_size)
    
    return train_loader, val_loader, test_loader

train_loader, val_loader, test_loader = split_data(dataset, batch_size=32)

In [4]:
# This class will be used to train the model on images only
class MultiImgsCNN(nn.Module):
    def __init__(self):
        super(MultiImgsCNN, self).__init__()
        # Define partitions, each with a convolutional layer and a ReLU activation function
        self.partition1 = nn.Sequential(nn.Conv2d(1, 128, kernel_size=3, padding=1), nn.ReLU()) 
        self.partition2 = nn.Sequential(nn.Conv2d(1, 128, kernel_size=3, padding=1), nn.ReLU())
        self.partition3 = nn.Sequential(nn.Conv2d(1, 128, kernel_size=3, padding=1), nn.ReLU())
        self.partition4 = nn.Sequential(nn.Conv2d(1, 128, kernel_size=3, padding=1), nn.ReLU())
        # passing each image through its partition helps to extract features from each image separately to make sure 
        
        # Define layers after combining the partitions
        self.combined_conv = nn.Sequential(nn.Conv2d(512, 256, kernel_size=3, padding=1),   # Example layer
        nn.MaxPool2d(kernel_size=2), 
        nn.ReLU(),
        nn.Conv2d(256, 128, kernel_size=3, padding=1),
        nn.MaxPool2d(kernel_size=2),
        nn.ReLU(),
        nn.Flatten(), #flatten the output of the previous layer to be able to pass it to the linear layer
        )
        self.fc = nn.Linear(128 * 16 * 16, 128)  # Adjust dimensions as necessary to match the input of next layer

    def forward(self, l):
        # Pass each image through its partition
        out1 = self.partition1(l[0])
        out2 = self.partition2(l[1])
        out3 = self.partition1(l[2])
        out4 = self.partition2(l[3])
    
        # Combine the outputs
        cnn_combined = torch.cat((out1, out2, out3, out4), 1)  # Concatenate along the channel dimension
        # Pass through the other layers
        cnn_combined = self.combined_conv(cnn_combined)
        cnn_combined = cnn_combined.view(cnn_combined.size(0), -1)  # Flatten the output of the previous layer to be able to pass it to the linear layer
        cnn_combined = self.fc(cnn_combined)
        
        return cnn_combined


In [5]:
# This class will be used to train the model on tabular(textual) features only
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True) #number of features, number of hidden units, number of layers
        self.fc_lstm = nn.Linear(hidden_size, 128)
        self.relu = nn.ReLU()

    def forward(self, textual_features):
        # Process textual features 
        lstm_out, _ = self.lstm(textual_features) 
        lstm_out = self.fc_lstm(lstm_out)
        lstm_out = self.relu(lstm_out)
        
        return lstm_out


In [6]:
# This class will be used to combine the output of the previous two models
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.images_layer = MultiImgsCNN()
        self.features_layer = LSTMModel(input_size=4, hidden_size=32, num_layers=2) # uncomment this when dropping the a column in the features df
        # self.features_layer = LSTMModel(input_size=3, hidden_size=32, num_layers=2) # uncomment this when dropping the a column in the features df
        
        # self.classifier = nn.Sequential(nn.Linear(128 + 128, 6)) # uncomment this when concatenating the output of the two models
        self.classifier = nn.Sequential(nn.Linear(128, 6)) # uncomment this when multiplying the output of the two models
    
    def forward(self, images, textual_features):
        image_model = self.images_layer(images)
        features_model = self.features_layer(textual_features)
        # x = torch.cat((image_model, features_model), dim=1) # uncomment this when concatenating the output of the two models
        x = image_model * features_model # uncomment this when multiplying the output of the two models
        # x = image_model + features_model # uncomment this when adding the output of the two models
        x = self.classifier(x) # pass the output of the two models to the classifier
        return x

In [7]:
# This function will be used to validate the model on the validation set and save the model with the best accuracy
def validate(model, val_loader, name):
    
    '''
    validation loop
    - check intermediate model performance
    - prints accuracy, returns accuracy
    '''
    
    val_loop = tqdm(val_loader, total = len(val_loader), leave = True) 
    num_correct = 0
    num_samples = 0
    
    # set the model for validation, gradients are NOT updated
    model.eval()

    with torch.no_grad():
        
        for x, y, z in val_loop:
            x = [tensor.to(device) for tensor in x]
            y = y.to(device=device)
            z = z.to(device=device)
            
            scores = model(x, y)
            # scores = model(x) # uncomment this when training on images only
            # scores = model(y) # uncomment this when training on tabular features only
            
            predictions = torch.argmax(scores, dim=1)
            num_correct += (predictions == z).sum() # calculate the number of correct predictions
            num_samples += predictions.size(0) # calculate the number of samples
            
     
    model.train() # set the model back to train mode 
    
    # save model
    PATH = f'{name}.pth'
    torch.save(model.state_dict(), PATH) 
    print('The model is saved!')
    
    
    return f'{float(num_correct)/float(num_samples)*100:.2f}'


In [8]:
# This function will be used to train the model
def train(model, criterion, optimizer, train_loader, validation_loader, num_epochs, name):
    
    '''
    training loop
    '''
    
    # set the model for training (with the gradient updates)
    model.train()
    
    # train for N epochs
    for epoch in range(num_epochs):
        
        loop = tqdm(train_loader, total = len(train_loader), leave = True) # progress bar for training loop
        
        # perform validation every 2 epochs 
        if epoch % 2 == 0 and epoch != 0:
            val_acc = validate(model, validation_loader, name)
        else:
            val_acc = 0
            
        for imgs, features, labels in loop:
                
            imgs = [img.to(device) for img in imgs]
            features = features.to(device)
            labels = labels.to(device)
                        
            outputs = model(imgs, features)
            # outputs = model(imgs) # uncomment this when training on images only
            # outputs = model(features) # uncomment this when training on tabular features only
            
            loss = criterion(outputs, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if val_acc != 0:
                loop.set_description(f'Epoch [{epoch+1}/{num_epochs}] validation_accuracy = {val_acc}') 
            else:
                loop.set_description(f'Epoch [{epoch+1}/{num_epochs}]')
            loop.set_postfix(loss = loss.item())
            
    print('Finished training!')

In [12]:
# calculate accuracy, precision, recall, f1 score for the test 
def test_model(model, loader):
    model.eval()
    test_accuracy = 0
    precision = 0
    recall = 0
    f1 = 0
    with torch.no_grad():
        for img, features, labels in loader:
            img = [tensor.to(device) for tensor in img]
            features = features.to(device)
            labels = labels.to(device)
            
            outputs = model(img, features)
        
            _, preds = torch.max(outputs, 1)
            # test_accuracy += accuracy(preds, labels)
            
            test_accuracy += accuracy_score(labels.cpu(), preds.cpu()) # we add the accuracy of each batch to the total accuracy to get the accuracy of the whole test set by
            precision += precision_score(labels.cpu(), preds.cpu(), average='macro')
            recall += recall_score(labels.cpu(), preds.cpu(), average='macro')
            f1 += f1_score(labels.cpu(), preds.cpu(), average='macro')
            
    print(f'Test Accuracy: {test_accuracy/len(test_loader)*100:.4f}% \n Test Precision: {precision/len(test_loader)*100:.4f}%, \n Test Recall: {recall/len(test_loader)*100:.4f}% \n Test F1 Score: {f1/len(test_loader)*100:.4f}%')
    
    return test_accuracy/len(test_loader), precision/len(test_loader), recall/len(test_loader), f1/len(test_loader)


In [21]:

model_mul = Model().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_mul.parameters(), lr=0.003)
epochs = 30
train(model_mul, criterion, optimizer, train_loader, val_loader, epochs, 'houses_mul')

/tmp/ipykernel_444835/2611406102.py:66: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  images = [torch.tensor(image) for image in images]
/tmp/ipykernel_444835/2611406102.py:70: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  textual_features = torch.tensor(textual_data)
/home/gussarayo@GU.GU.SE/.venv/lib64/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/tmp/ipykernel_444835/2611406102.py:66: UserWarning: To copy construct from a tensor, it is recommended to

Test Accuracy: 90.6250% 
 Test Precision: 71.3281%, 
 Test Recall: 66.6667% 
 Test F1 Score: 66.7960%


/home/gussarayo@GU.GU.SE/.venv/lib64/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/tmp/ipykernel_444835/2611406102.py:66: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  images = [torch.tensor(image) for image in images]
/tmp/ipykernel_444835/2611406102.py:70: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  textual_features = torch.tensor(textual_data)


In [16]:
model_imgs = MultiImgsCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_imgs.parameters(), lr=0.003)
epochs = 30
train(model_imgs, criterion, optimizer, train_loader, val_loader, epochs, 'houses_imgs')

  0%|          | 0/11 [00:00<?, ?it/s]/tmp/ipykernel_565101/990838682.py:65: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  images = [torch.tensor(image) for image in images]
/tmp/ipykernel_565101/990838682.py:69: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  textual_features = torch.tensor(textual_data)
100%|██████████| 3/3 [00:02<00:00,  1.50it/s]


The model is saved!


100%|██████████| 3/3 [00:01<00:00,  1.53it/s]


The model is saved!


100%|██████████| 3/3 [00:01<00:00,  1.57it/s]


...

The model is saved!


100%|██████████| 3/3 [00:01<00:00,  1.57it/s]


The model is saved!


Epoch [30/30]: 100%|██████████| 11/11 [00:07<00:00,  1.43it/s, loss=0.279] 

Finished training!


In [18]:
acc_imgs, prec_imgs, rec_imgs, f1_imgs = test_model(model_imgs, test_loader)

/tmp/ipykernel_565101/990838682.py:65: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  images = [torch.tensor(image) for image in images]
/tmp/ipykernel_565101/990838682.py:69: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  textual_features = torch.tensor(textual_data)
/home/gussarayo@GU.GU.SE/.venv/lib64/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/tmp/ipykernel_565101/990838682.py:65: UserWarning: To copy construct from a tensor, it is recommended to use source

Test Accuracy: 37.5000% 
 Test Precision: 27.2240%, 
 Test Recall: 28.7054% 
 Test F1 Score: 22.9342%


/home/gussarayo@GU.GU.SE/.venv/lib64/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/gussarayo@GU.GU.SE/.venv/lib64/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/tmp/ipykernel_565101/990838682.py:65: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  images = [torch.tensor(image) for image in images]
/tmp/ipykernel_565101/990838682.py:69: UserWarning: To copy construct from a t

In [11]:
model_feat = LSTMModel(input_size=4, hidden_size=32, num_layers=2).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_feat.parameters(), lr=0.003)
epochs = 30
train(model_feat, criterion, optimizer, train_loader, val_loader, epochs, 'houses_feat')

  0%|          | 0/11 [00:00<?, ?it/s]/tmp/ipykernel_566259/990838682.py:65: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  images = [torch.tensor(image) for image in images]
/tmp/ipykernel_566259/990838682.py:69: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  textual_features = torch.tensor(textual_data)
100%|██████████| 3/3 [00:01<00:00,  1.69it/s]


The model is saved!


100%|██████████| 3/3 [00:01<00:00,  1.75it/s]


The model is saved!


100%|██████████| 3/3 [00:01<00:00,  1.75it/s]


...

The model is saved!


100%|██████████| 3/3 [00:01<00:00,  1.74it/s]


The model is saved!


Epoch [30/30]: 100%|██████████| 11/11 [00:07<00:00,  1.45it/s, loss=0.0979]

Finished training!


In [13]:
acc_feat, prec_feat, rec_feat, f1_feat = test_model(model_feat, test_loader)

/tmp/ipykernel_566259/990838682.py:65: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  images = [torch.tensor(image) for image in images]
/tmp/ipykernel_566259/990838682.py:69: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  textual_features = torch.tensor(textual_data)
/home/gussarayo@GU.GU.SE/.venv/lib64/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/tmp/ipykernel_566259/990838682.py:65: UserWarning: To copy construct from a tensor, it is recommended to us

Test Accuracy: 87.5000% 
 Test Precision: 56.2500%, 
 Test Recall: 62.5000% 
 Test F1 Score: 59.0476%


/home/gussarayo@GU.GU.SE/.venv/lib64/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/tmp/ipykernel_566259/990838682.py:65: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  images = [torch.tensor(image) for image in images]
/tmp/ipykernel_566259/990838682.py:69: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  textual_features = torch.tensor(textual_data)


In [11]:
model_60 = Model().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_60.parameters(), lr=0.003)
epochs = 60
train(model_60, criterion, optimizer, train_loader, val_loader, epochs, 'houses_60')

  0%|          | 0/11 [00:00<?, ?it/s]/tmp/ipykernel_596068/990838682.py:65: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  images = [torch.tensor(image) for image in images]
/tmp/ipykernel_596068/990838682.py:69: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  textual_features = torch.tensor(textual_data)
100%|██████████| 3/3 [00:02<00:00,  1.45it/s]


The model is saved!


100%|██████████| 3/3 [00:01<00:00,  1.50it/s]


The model is saved!


100%|██████████| 3/3 [00:02<00:00,  1.46it/s]


...

The model is saved!


100%|██████████| 3/3 [00:03<00:00,  1.01s/it]


The model is saved!


Epoch [60/60]: 100%|██████████| 11/11 [00:09<00:00,  1.15it/s, loss=2.11e-6]

Finished training!


In [14]:
acc_60, prec_60, rec_60, f1_60 = test_model(model_60, test_loader)

/tmp/ipykernel_596068/990838682.py:65: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  images = [torch.tensor(image) for image in images]
/tmp/ipykernel_596068/990838682.py:69: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  textual_features = torch.tensor(textual_data)


/tmp/ipykernel_596068/990838682.py:65: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  images = [torch.tensor(image) for image in images]
/tmp/ipykernel_596068/990838682.py:69: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  textual_features = torch.tensor(textual_data)
/tmp/ipykernel_596068/990838682.py:65: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  images = [torch.tensor(image) for image in images]
/tmp/ipykernel_596068/990838682.py:69: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() o

Test Accuracy: 92.1875% 
 Test Precision: 86.1925%, 
 Test Recall: 80.7229% 
 Test F1 Score: 81.8941%


/tmp/ipykernel_596068/990838682.py:65: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  images = [torch.tensor(image) for image in images]
/tmp/ipykernel_596068/990838682.py:69: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  textual_features = torch.tensor(textual_data)
